In [1]:
#!/usr/local/bin/env python
from __future__ import division
import numpy as np
import pandas as pd
from lmfit import Parameters, Minimizer, report_fit, fit_report
from scipy.optimize import minimize
from radd.utils import *
from radd.cRADD import recost
from radd import build
from radd.fitre import rangl_re

In [2]:
grouped.apply()

NameError: name 'grouped' is not defined

In [6]:
grouped = data.groupby(['idx', 'Cond']).apply(rangl_re, cutoff=.5)
#grouped = data.apply(rangl_data(kind='reactive'))
#cdf.unstack().values.shape
#[rangl_re(idf) for ix, idf in cdf.groupby('ix')]

In [7]:
np.vstack(grouped.as_matrix()).shape

(122, 16)

In [17]:
cdf

idx  Cond
28   bsl     [0.991736, 1.0, ...
     pnl     [0.975207, 1.0, ...
29   bsl     [0.991736, 1.0, ...
     pnl     [0.966942, 1.0, ...
30   bsl     [0.942149, 1.0, ...
     pnl     [0.867769, 1.0, ...
31   bsl     [1.0, 1.0, 1.0, ...
     pnl     [0.966942, 1.0, ...
32   bsl     [0.950413, 1.0, ...
     pnl     [0.966942, 1.0, ...
33   bsl     [0.942149, 1.0, ...
     pnl     [0.909091, 1.0, ...
34   bsl     [0.950413, 1.0, ...
...
86   pnl     [0.884298, 1.0, ...
87   bsl     [1.0, 1.0, 1.0, ...
     pnl     [1.0, 1.0, 1.0, ...
88   bsl     [0.809917, 1.0, ...
     pnl     [0.719008, 1.0, ...
89   bsl     [0.975207, 1.0, ...
     pnl     [0.958678, 1.0, ...
90   bsl     [0.991736, 1.0, ...
     pnl     [0.933884, 1.0, ...
95   bsl     [1.0, 1.0, 0.9, ...
     pnl     [0.975207, 1.0, ...
97   bsl     [1.0, 1.0, 1.0, ...
     pnl     [0.958678, 0.95,...
Length: 122, dtype: object

In [14]:
cdf['bsl']

array([ 0.94201326,  0.99426228,  0.98278689,  0.8967213 ,  0.50491804,
        0.10327869,  5.05134583,  5.32041454,  5.58484221,  5.85082388,
        6.24409151,  4.91785669,  5.18539333,  5.45045137,  5.58876324,
        5.9842329 ], dtype=float32)

In [5]:
data = redf_store['redata']
inits = redf_store['rebsl_boot_popt']
inits

{'a': 0.44409409212000012,
 'ssv': 0.94733930094933305,
 'tr': 0.30487069828933316,
 'v': 1.1223597714793327,
 'z': 0.15424655465199996}

In [ ]:

def fit_reactive_model(y, inits={}, depends=['xx'], wts=None, model='radd', ntrials=5000, maxfun=5000, ftol=1.e-3, xtol=1.e-3, all_params=0, disp=False):


      lim = {'tr': (.001, .5), 'v': (.01, 4.),  'ssv': (-4., -.01),  'a' : (.01, .6)}
      if 'pGo' in inits.keys(): del inits['pGo']
      if 'ssd' in inits.keys(): del inits['ssd']

      p=Parameters()
      if all_params:
            p.add('a', value=inits['a'], vary=1, min=lim['a'][0], max=lim['a'][1])
            p.add('ssv', value=-abs(inits['ssv']), vary=1, min=lim['ssv'][0], max=lim['ssv'][1])
            p.add('v', value=inits['v'], vary=1, min=lim['v'][0], max=lim['v'][1])
            p.add('zperc', value=inits['z']/inits['a'], vary=1, min=.001, max=.99)
            p.add('tr', value=inits['tr'], vary=1, min=lim['tr'][0], max=lim['tr'][1])
            p.add('z', expr="zperc*a")
      else:
            for key, val in inits.items():
                  if key in depends:
                        p.add(key, value=val, vary=1, min=lim[key][0], max=lim[key][1])
                        continue
                  p.add(key, value=val, vary=all_params)

      popt = Minimizer(recost, p, fcn_args=(y, ntrials), fcn_kws={'wts':wts}, method='Nelder-Mead')
      popt.fmin(maxfun=maxfun, ftol=ftol, xtol=xtol, full_output=True, disp=disp)
      params={k: p[k].value for k in p.keys()}
      params['chi']=popt.chisqr
      resid=popt.residual; yhat=y+resid
      return params, yhat



In [24]:
all_params=1
lim = {'tr': (.001, .5), 'v': (.01, 4.),  'ssv': (-4., -.01),  'a' : (.01, .6)}
lim = {'tr': (.001, .5), 'v': (.01, 4.),  'ssv': (-4., -.01),  'a' : (.01, .6)}
if 'pGo' in inits.keys(): del inits['pGo']
if 'ssd' in inits.keys(): del inits['ssd']

p=Parameters()
if all_params:
      p.add('a', value=inits['a'], vary=1, min=lim['a'][0], max=lim['a'][1])
      p.add('ssv', value=-abs(inits['ssv']), vary=1, min=lim['ssv'][0], max=lim['ssv'][1])
      p.add('v', value=inits['v'], vary=1, min=lim['v'][0], max=lim['v'][1])
      #p.add('zperc', value=inits['z']/inits['a'], vary=1, min=.001, max=.99)
      p.add('tr', value=inits['tr'], vary=1, min=lim['tr'][0], max=lim['tr'][1])
      #p.add('z', expr="zperc*a")
      p.add('z', value=inits['z'], vary=1, min=.01, max=.3)

In [ ]:
#print lmfit.Minimizer(recost, p, fcn_args=(y, ntrials), fcn_kws=np.ones_like(y), )


In [18]:
m = build.Model(data=data, inits=inits, all_params=1, depends_on={'v':'Cond'}, fit='subjects')
m.prepare_fit()

preparing reactive model using subjects method


In [29]:
ntrials=2000
y=utils.rangl_re(data)
popt = Minimizer(recost, p, fcn_args=(y, ntrials), fcn_kws={'wts':m.wts['bsl']})
popt.scalar_minimize(maxfun=50, full_output=True, disp=True, method='differential_evolution')


differential_evolution step 1: f(x)= 0.16625
differential_evolution step 2: f(x)= 0.16625
differential_evolution step 3: f(x)= 0.16625
differential_evolution step 4: f(x)= 0.16625
differential_evolution step 5: f(x)= 0.16625
differential_evolution step 6: f(x)= 0.16625
differential_evolution step 7: f(x)= 0.16625
differential_evolution step 8: f(x)= 0.016747
differential_evolution step 9: f(x)= 0.016747
differential_evolution step 10: f(x)= 0.016747
differential_evolution step 11: f(x)= 0.016747
differential_evolution step 12: f(x)= 0.016747
differential_evolution step 13: f(x)= 0.016747
differential_evolution step 14: f(x)= 0.016747
differential_evolution step 15: f(x)= 0.016747
differential_evolution step 16: f(x)= 0.016747
differential_evolution step 17: f(x)= 0.016747
differential_evolution step 18: f(x)= 0.016747
differential_evolution step 19: f(x)= 0.016747
differential_evolution step 20: f(x)= 0.016747
differential_evolution step 21: f(x)= 0.016747
differential_evolution step 2

In [37]:
p.pretty_print()

Parameters({
    'a': <Parameter 'a', 0.44409409212000012, bounds=[0.01:0.6]>, 
    'ssv': <Parameter 'ssv', -0.94733930094933305, bounds=[-4.0:-0.01]>, 
    'v': <Parameter 'v', 1.1223597714793327, bounds=[0.01:4.0]>, 
    'tr': <Parameter 'tr', 0.30487069828933316, bounds=[0.001:0.5]>, 
    'z': <Parameter 'z', 0.15424655465199996, bounds=[0.01:0.3]>, 
    })



In [45]:
popt.values


ValuesView(Parameters([('a', <Parameter 'a', 0.44409409212000012, bounds=[0.01:0.6]>), ('ssv', <Parameter 'ssv', -0.94733930094933305, bounds=[-4.0:-0.01]>), ('v', <Parameter 'v', 1.1223597714793327, bounds=[0.01:4.0]>), ('tr', <Parameter 'tr', 0.30487069828933316, bounds=[0.001:0.5]>), ('z', <Parameter 'z', 0.15424655465199996, bounds=[0.01:0.3]>)]))

In [40]:
print fit_report(popt)

[[Fit Statistics]]
    # function evals   = 0
    # data points      = 0
    # variables        = unknown
    chi-square         = unknown
    reduced chi-square = unknown
[[Variables]]
    a:     0.44409409 (init= 0.4440941)
    ssv:  -0.94733930 (init=-0.9473393)
    v:     1.12235977 (init= 1.12236)
    tr:    0.30487069 (init= 0.3048707)
    z:     0.15424655 (init= 0.1542466)
[[Correlations]] (unreported correlations are <  0.100)


In [ ]:
fit = residual(fit_params, x)
print(fit_report(fit_params))
with open('fit_report.txt', 'w') as f:
      f.write(print(fit_report(p)))
      f.write(print(fit_report(popt)))


In [47]:
ntrials=2000
y=utils.rangl_re(data)
popt2 = Minimizer(recost, p, fcn_args=(y, ntrials), fcn_kws={'wts':m.wts['bsl']})
popt2.minimize(maxfun=5, full_output=True, disp=True, maxiter=5, method='differential_evolution')


differential_evolution step 1: f(x)= 0.767601
differential_evolution step 2: f(x)= 0.767601
differential_evolution step 3: f(x)= 0.206745
differential_evolution step 4: f(x)= 0.206745
differential_evolution step 5: f(x)= 0.123377


In [52]:
fit_report(popt2)

'[[Fit Statistics]]\n    # function evals   = 0\n    # data points      = 0\n    # variables        = unknown\n    chi-square         = unknown\n    reduced chi-square = unknown\n[[Variables]]\n    a:     0.44409409 (init= 0.4440941)\n    ssv:  -0.94733930 (init=-0.9473393)\n    v:     1.12235977 (init= 1.12236)\n    tr:    0.30487069 (init= 0.3048707)\n    z:     0.15424655 (init= 0.1542466)\n[[Correlations]] (unreported correlations are <  0.100)'

In [55]:

minimize(recost, p, args=(y, ntrials), kws={'wts':m.wts['bsl']}, fit_kws={'maxiter':5, 'full_output':True, 'disp':True},  method='differential_evolution')

TypeError: minimize() got an unexpected keyword argument 'fcn_kws'